In [1]:
import csv
import os
from scenedetect import detect, ContentDetector, split_video_ffmpeg

# Set the base name
base_name = "Cigarettes"

# Construct file and directory names
video_file = f'{base_name}.mp4'
csv_filename = f'{base_name}_scene_detect.csv'
output_directory = f'{base_name}_split_videos'

os.makedirs(output_directory, exist_ok=True)

def get_scene_info(scene):
    start_frame, start_timecode = int(scene[0].get_frames()), scene[0].get_timecode()
    end_frame, end_timecode = int(scene[1].get_frames()), scene[1].get_timecode()

    # Convert timecode strings to frames
    start_timecode_frames = int(scene[0].get_frames())
    end_timecode_frames = int(scene[1].get_frames())

    # Calculate lengths
    length_frames, length_timecode, length_seconds = end_frame - start_frame, end_timecode_frames - start_timecode_frames, scene[0].get_seconds()

    return start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode, scene[0].get_seconds(), scene[1].get_seconds(), length_seconds

with open(csv_filename, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Video Name', 'Scene Number', 'Start Frame', 'End Frame', 'Length (frames)', 'Start Timecode', 'End Timecode', 'Length (timecode)', 'Start Time (seconds)', 'End Time (seconds)', 'Length (seconds)'])

    for i, scene in enumerate(detect(video_file, ContentDetector())):
        split_video_name = f'{base_name}-Scene-{i + 1:03d}.mp4'
        print(f'Splitting scene {i + 1}: {split_video_name}')
        split_video_ffmpeg(video_file, [scene], output_file_template=os.path.join(output_directory, split_video_name), show_progress=True)

        start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode, start_seconds, end_seconds, length_seconds = get_scene_info(scene)
        csv_writer.writerow([split_video_name, i + 1, start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode, start_seconds, end_seconds, length_seconds])

print(f'CSV file "{csv_filename}" has been created.')
print(f'Split videos are saved in the "{output_directory}" directory.')


c:\Users\poohm\.conda\envs\scene-detect\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Splitting scene 1: Cigarettes-Scene-001.mp4


100%|██████████| 222/222 [00:00<00:00, 246.15frame/s]


Splitting scene 2: Cigarettes-Scene-002.mp4


100%|██████████| 233/233 [00:01<00:00, 198.18frame/s]


...

100%|██████████| 184/184 [00:01<00:00, 137.97frame/s]


Splitting scene 43: Cigarettes-Scene-043.mp4


100%|██████████| 109/109 [00:00<00:00, 165.63frame/s]


Splitting scene 44: Cigarettes-Scene-044.mp4


  0%|          | 0/204 [00:00<?, ?frame/s]